In [73]:
import pandas as pd
import numpy as np
from datetime import date
from scipy.sparse import coo_matrix
import requests

data_df = pd.read_csv('./../data/ratings.csv', sep=',',header=0, names=["UserID", "MovieID", "Rating", "Timestamp"],dtype={'Rating':'float64', "Timestamp":"float64"})
meta_df = pd.read_csv('./../data/movies_metadata.csv',header=0, usecols=["id", "imdb_id", "original_title"])
data_df["Rating"] = (data_df["Rating"] > 0).astype(float)

In [69]:
## we want the k movies that have been rated the most times.
data_df.dropna()
k = 100

value_counts = data_df['MovieID'].value_counts()
movies_in_meta = meta_df["id"].unique()
print(value_counts)
movies = []
for movie, _ in value_counts.iteritems():
    if(len(movies)>=k):
        break
    if(str(movie) in movies_in_meta):
        movies.append(meta_df[meta_df["id"] == str(movie)]["original_title"].item())


356       91921
318       91082
296       87901
593       84078
2571      77960
          ...  
151011        1
176257        1
167416        1
134633        1
123324        1
Name: MovieID, Length: 45115, dtype: int64


In [75]:
api_results = []
IMDB_API_KEY_HERE = "get ur own ;)"
imdb_url = "https://imdb-api.com/en/API/SearchMovie/"+IMDB_API_KEY_HERE+"/"
for movie in movies:
    mov_url = imdb_url + movie
    res = requests.get(mov_url)
    api_results.append(res.json)

In [86]:
images = []
for result in api_results:
    try:
        res = result()
        # print(res['results'][0]['image'])
        images.append(res['results'][0]['image'])
    except:
        images.append("not found")

In [78]:
api_results[0]()

{'searchType': 'Movie',
 'expression': 'The Million Dollar Hotel',
 'results': [{'id': 'tt0120753',
   'resultType': 'Title',
   'image': 'https://imdb-api.com/images/original/MV5BNDQ2MGUwYjYtODFiNy00OTViLWJhOTgtZWNmM2I2MDRmMzQ1XkEyXkFqcGdeQXVyNjU0NTI0Nw@@._V1_Ratio0.7273_AL_.jpg',
   'title': 'The Million Dollar Hotel',
   'description': '(2000)'},
  {'id': 'tt0172464',
   'resultType': 'Title',
   'image': 'https://imdb-api.com/images/original/MV5BODY2MzUxMjczMF5BMl5BanBnXkFtZTYwOTc0NzI5._V1_Ratio0.7273_AL_.jpg',
   'title': 'The Million Dollar Kid',
   'description': '(2000)'},
  {'id': 'tt0066728',
   'resultType': 'Title',
   'image': 'https://imdb-api.com/images/original/MV5BYWQyYTU0ZmQtNDE5ZS00M2E4LWIxZjYtOWI5NTlhOWVlYWJkL2ltYWdlL2ltYWdlXkEyXkFqcGdeQXVyNzc5MjA3OA@@._V1_Ratio0.7273_AL_.jpg',
   'title': 'The Million Dollar Duck',
   'description': '(1971)'},
  {'id': 'tt0037073',
   'resultType': 'Title',
   'image': 'https://imdb-api.com/images/original/MV5BOGVkN2QyZjEtMDMyMy00Z

In [93]:
with open('top100images.json','w', encoding="utf-8") as file:
    file.write("{\n")
    for i in range(100):
        file.write('"')
        file.write(movies[i])
        file.write('"')
        file.write(':')
        file.write('"')
        file.write(images[i])
        file.write('"')
        if (i!=99):
            file.write(',')
        file.write('\n')
    file.write("}\n")